In [1]:
import numpy as np
import pandas as pd

In [147]:
df = pd.read_csv('../static/data/movie01.csv')
# df2 = pd.read_csv('../static/data/test02.csv')
# print(df.shape, df2.shape)
# df = pd.concat([df, df2])
# df.set_index('code', inplace=True)
# df.reset_index(inplace=True)
print(df.shape)
df.head(1)


(2673, 10)


,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
0,20232561,몸값part.2 (Bargain part.2),스릴러,장편 | 일반영화 | 스릴러 | 105분 13초 | 청소년관람불가 | 한국,2023-07-05,2022년,전우성,NaN,https://www.kobis.or.kr/common/mast/movie/2023...,"무너진 세상, 폭발하는 광기\n미친 자들의 목숨을 건 거래가 시작된다!\n\n서로의..."


In [148]:
m_type, m_kind, m_genre, m_time, m_rated, m_nation = [], [], [], [], [], []
for info in df.summering.values:
    infos = info.split('|')
    infos_len = len(infos)
    m_type.append(infos[0].strip())                     # 장편
    m_kind.append(infos[1].strip())                     # 일반영화, 예술영화..
    m_genre.append(infos[2].strip().replace(',', ''))   # 애니메이션, 드라마, 멜로
    m_nation.append(infos[-1].strip())                  # 한국
    
    # 요약 정보 개수가 다를 수 있다
    if infos_len == 6:
        m_time.append(infos[-3].strip())                # 70분
        m_rated.append(infos[-2].strip())
    elif infos_len == 5 :
        m_time.append(infos[-2].strip())               # 12세 이상 관람가
        m_rated.append('')
    elif infos_len == 4:
        m_rated.append('')
        m_time.append('')
    else:
        print(info)

len(m_type)

2673

In [149]:
df['m_type'] = m_type
df['m_kind'] = m_kind
df['m_genre'] = m_genre
df['m_nation'] = m_nation
df['m_time'] = m_time
df['m_rated'] = m_rated

In [150]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
0,20232561,몸값part.2 (Bargain part.2),스릴러,장편 | 일반영화 | 스릴러 | 105분 13초 | 청소년관람불가 | 한국,2023-07-05,2022년,전우성,NaN,https://www.kobis.or.kr/common/mast/movie/2023...,"무너진 세상, 폭발하는 광기\n미친 자들의 목숨을 건 거래가 시작된다!\n\n서로의...",장편,일반영화,스릴러,한국,105분 13초,청소년관람불가


In [152]:
df.isna().sum()

code                  0
title                 0
genre                 0
summering             0
first_day             0
production_year       0
movie_director      768
star_actor         2673
img                   0
synopsis              0
m_type                0
m_kind                0
m_genre               0
m_nation              0
m_time                0
m_rated               0
dtype: int64

In [151]:
df.head(2000).tail(5)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
1995,20227635,라스트 플라이트 (The Last Flight),기타,장편 | 일반영화 | 기타 | 94분 20초 | 프랑스,해당정보없음,2009년,카림 드리디,NaN,https://www.kobis.or.kr/common/mast/movie/2022...,사랑하는 연인을 찾아 헤매는 비행사와 영국의 식민지배 방식을 못마땅히 여겼던 군인의...,장편,일반영화,기타,프랑스,94분 20초,
1996,20226777,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교 (Crayon Shin-cha...,애니메이션,장편 | 일반영화 | 애니메이션 | 104분 37초 | 전체관람가 | 일본,2022-09-28,2021년,타카하시 와타루,NaN,https://www.kobis.or.kr/common/mast/movie/2022...,"짱구와 친구들, 명문학교에 입학하다?!\n\n명문 엘리트 양성학교에 체험 입학하게 ...",장편,일반영화,애니메이션,일본,104분 37초,전체관람가
1997,20227392,자드 - 왓 어 뷰티풀 모먼트 투어 (ZARD - What a beautiful m...,공연,장편 | 일반영화 | 공연 | 101분 30초 | 전체관람가 | 일본,2022-09-28,2022년,NaN,NaN,https://www.kobis.or.kr/common/mast/movie/2022...,일본 밴드 ZARD의 마지막 전국 투어,장편,일반영화,공연,일본,101분 30초,전체관람가
1998,20226884,나의 여신,기타,장편 | 독립영화 | 기타 | 86분 18초 | 12세이상관람가 | 한국,해당정보없음,2021년,NaN,NaN,https://www.kobis.or.kr/kobis/web/comm/images/...,교수가 되기 위해 노력중인 무속 연구자가 연구를 위해 제주도의 한 젊은 무당을 찾아...,장편,독립영화,기타,한국,86분 18초,12세이상관람가
1999,20193128,가나안 김용기,다큐멘터리(1),"장편 | 예술영화 | 다큐멘터리, 드라마 | 91분 59초 | 12세이상관람가 | 한국",해당정보없음,2022년,김상철,NaN,https://www.kobis.or.kr/common/mast/movie/2021...,가나안농군학교 설립자 김용기 장로의 이야기.,장편,예술영화,다큐멘터리 드라마,한국,91분 59초,12세이상관람가


In [153]:
df.fillna('', inplace=True)

In [154]:

df[df.movie_director.str.find('[주연]') >= 0]

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated


In [81]:
# 감독이 없고 주연 배우만 있는 경우가 있어서 잘 못 들어갔다. 
# 배우에 전처리 해서 넣어 주었다.
indexs = df[df.movie_director.str.find('[주연]') >= 0].index
for index in indexs:
    df.iloc[index, 7] = df.iloc[index, 6].replace('[주연]', '').strip()
    df.iloc[index, 6] = ''

In [82]:
df[df.movie_director.str.find('[주연]') >=0 ]

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated


In [160]:
# 감독 자리에 다른 값들이 들어갔다. 주연 배우 자리에는 데이터가 없고
# 잘못 처리했다. 일단 감독이 이상한 것은 지우고 테스트
indexs = df[df.movie_director.str.find('[') >= 0].index
indexs

Index([  33,   39,  122,  125,  126,  127,  128,  129,  130,  131,
       ...
       2383, 2401, 2408, 2411, 2420, 2449, 2468, 2568, 2606, 2642],
      dtype='int64', length=207)

In [161]:
df.drop(df.index[indexs], inplace=True)
df.set_index('code', inplace=True)
df.reset_index(inplace=True)

In [162]:
df.shape

(2466, 16)

In [84]:
for index in df[df.movie_director.str.find('[') >= 0].index:
    df.iloc[index, 6] = ''

In [87]:
df.to_csv('../static/data/movie_test.csv', index=False)

In [89]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated
0,20232491,핑크퐁 시네마 콘서트3 : 진저브레드맨을 잡아라 (Pinkfong Sing-Alon...,애니메이션,장편 | 일반영화 | 애니메이션 | 65분 14초 | 전체관람가 | 한국,2023-07-26,2023년,변희선,조경이(핑크퐁) | 이소은(호기) | 박요한(진저브레드맨) | 김새해(제니) | 장...,https://www.kobis.or.kr/common/mast/movie/2023...,“세상에서 가장 맛있는 아이스크림을 먹고 싶어!”\r\n\r\n아기상어가 원하는 최...,장편,일반영화,애니메이션,한국,65분 14초,전체관람가


In [109]:
text = df.synopsis.values[40:50]
text

array(["“저는 사랑이 뭔지 잘 몰라요”\r\n“너는 몰라도 돼, 우리가 사랑이라고 생각하니까”\r\n\r\n사랑 빼곤 모든 것이 충족된 삶을 살던 도쿄 유명 패션 잡지 에디터 ‘료스케’(스즈키 료헤이)는\r\n삶은 고되고 사랑은 두려운 퍼스널 트레이너 '류타'(미야자와 히오)를 만난다.\r\n운명처럼 찾아온 사랑에 모든 날들이 특별해지고,\r\n서로를 만나기 전과는 비교할 수 없을 정도로 행복한 나날을 보내는 두 남자.\r\n평소와 다름없던 어느 날\r\n영원할 것 같던 이들에게도 이별이 찾아오는데…\r\n\r\n사랑이 필요하지만 사랑이 두려운 두 남자,\r\n이별 후 비로소 사랑은 시작된다.",
       '집이 없어 비닐하우스에 기거하지만 하루하루 성실하게 살아가고 있던 요양보호사 문정이 갑작스럽게 비극적인 사건을 겪으면서 일어나는 이야기',
       '평생 100일의 연애기간을 넘기지 못하는 하윤\r\n그런 하윤이 운명의 남자라고 생각했던 태령마저 이별을 말한다.\r\n\r\n100일을 넘기기 위해 한달간의 유예기간을 가지자고 한\r\n하윤의 제안을 태령은 승낙한다.\r\n\r\n이 과정에서 하윤은 자신의 오랜 남사친인 강진에게 질투작전을 위해\r\n가짜 썸남인척 연기해달라고 부탁한다.\r\n\r\n두 남자와 이중계약을 맺어버린 하윤은 한 달동안 두 남자 사이에서 갈등한다.',
       '주목받는 젊은 작가 재이는 신작 출간을 앞두고 있다. 재이의 동거인이자 애인인 건우는 보습 학원 영어 강사로 일하며 묵묵하게 재이를 보조하고 있다. 두 사람은 모두 서로를 아끼고 있으며 이대로 행복하다. 하지만 갑자기 예정하지 않았던 아이가 생기면서 두 사람의 삶에 크나큰 변화와 균열이 생기기 시작한다.',
       '수많은 스캔들과 사고로 인해 추락에 추락을 거듭한 왕년의 슈퍼스타 박혁빈\r\n어느날. 그가 진행하는 라디오 방송에 여러 페널들이 모이게 되고,\r\n이곳으로 보낸 여러사연들을 읽으면서 패널들과의 인연들이 나타나게 되고 새로운 사건이 시

In [93]:
import string, re
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [163]:
# 시놉시스전처리
texts = df.synopsis.apply(lambda x: re.sub('['+string.punctuation+'\r\n“”‘’…]', ' ', x))

In [164]:
from konlpy.tag import Okt

In [165]:
okt = Okt()
# 시놉시스 전처리의 형태소 분석
df['morphs'] = [' '.join(okt.morphs(text)) for text in texts]

In [166]:
df.head(1)

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis,m_type,m_kind,m_genre,m_nation,m_time,m_rated,morphs
0,20232561,몸값part.2 (Bargain part.2),스릴러,장편 | 일반영화 | 스릴러 | 105분 13초 | 청소년관람불가 | 한국,2023-07-05,2022년,전우성,,https://www.kobis.or.kr/common/mast/movie/2023...,"무너진 세상, 폭발하는 광기\n미친 자들의 목숨을 건 거래가 시작된다!\n\n서로의...",장편,일반영화,스릴러,한국,105분 13초,청소년관람불가,무너진 세상 폭발 하는 광기 미친 자 들 의 목숨 을 건 거래 가 시작 된다 서로 ...


In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvect = TfidfVectorizer(stop_words='english')
total_tv = tvect.fit_transform(df.morphs)
total_tv.shape

(2466, 24271)

In [168]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')
total_cv = cvect.fit_transform(df.morphs)
total_cv.shape

(2466, 24271)

In [169]:
indices = pd.Series(df.index, index=df.code)

In [187]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(total_tv)
cosine_sim.shape

(2466, 2466)

In [188]:
cosine_sim_cv = cosine_similarity(total_cv)
cosine_sim_cv.shape

(2466, 2466)

In [191]:
def get_recommendation(code, cos_sim):
    index = indices[code]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.code.iloc[movie_indices], df.title.iloc[movie_indices]

In [190]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total = linear_kernel(total_tv, total_tv)
cosine_sim_total.shape

(2466, 2466)

In [184]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_total_cv = linear_kernel(total_cv, total_cv)

In [ ]:
df.tail(5)

In [197]:
get_recommendation('20225534', cosine_sim_cv)

(1131    20020064
 2372    20225570
 1451    20060363
 2220    20226279
 2221    20021081
 1090    20156102
 1692    20211328
 1767    20060006
 1212    20229453
 2337    20226003
 Name: code, dtype: object,
 1131                    잔다라 (Jan Dara)
 2372    눈감은 여름 (Summer In Closed Eyes)
 1451            디어 평양 (Dear Pyongyang)
 2220                  다라트 (Dry Season)
 2221                  아부나 (Our Father)
 1090     바닷마을 다이어리 (Our Little Sister)
 1692    검객: 척살금의위 (Chivalrous Forever)
 1767        메종 드 히미코 (Mezon Do Himiko)
 1212                   문라이즈 (Moonrise)
 2337         달을 빚는 토끼 (Dumpling Bunny)
 Name: title, dtype: object)

In [192]:
get_recommendation('20225534', cosine_sim)

(2372    20225570
 1090    20156102
 1131    20020064
 1451    20060363
 2220    20226279
 2081    20212704
 1351    20228974
 2221    20021081
 627     20227527
 1028    20228892
 Name: code, dtype: object,
 2372                       눈감은 여름 (Summer In Closed Eyes)
 1090                        바닷마을 다이어리 (Our Little Sister)
 1131                                       잔다라 (Jan Dara)
 1451                               디어 평양 (Dear Pyongyang)
 2220                                     다라트 (Dry Season)
 2081    뉴타운 생존자 수색작전 (Search operation: The New Town S...
 1351                                     재일배이트 (Jailbait)
 2221                                     아부나 (Our Father)
 627                                         공작새 (Peafowl)
 1028                                      마에스트로 (Maestro)
 Name: title, dtype: object)

In [199]:
# for code in df.code.values:
code = "20225534"
sim_scores = pd.Series(cosine_sim[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

2372    0.157410
1090    0.141053
1131    0.133918
1451    0.132550
2220    0.116664
2081    0.115052
1351    0.113074
2221    0.111055
627     0.100726
dtype: float64

In [198]:
code = "20225534"
sim_scores = pd.Series(cosine_sim_cv[indices[code]])
score = sim_scores.sort_values(ascending=False).head(10).tail(9)
score

1131    0.311990
2372    0.311400
1451    0.287034
2220    0.283731
2221    0.277746
1090    0.267632
1692    0.247717
1767    0.246008
1212    0.228218
dtype: float64